## Introducción
Simulamos un sistema con `n` máquinas operativas y `s` repuestos para estimar el tiempo promedio hasta el colapso del sistema.

In [66]:
from simulator import SystemSimulator
import numpy as np
import heapq
import matplotlib.pyplot as plt
from scipy.special import erfcinv

## Configuración inicial

In [68]:
# Configurar parámetros
n = 5       # Máquinas operativas
s = 2       # Repuestos
lambda_fail = 0.1  # Tasa de fallos
mu_repair = 0.5    # Tasa de reparación
k = 100    # Número de simulaciones iniciales
c=erfcinv(0.05)
d = 0.5

# Definir distribuciones
F_dist = lambda size=None: np.random.exponential(1/lambda_fail, size)
G_dist = lambda: np.random.exponential(1/mu_repair)

## Ejecución de la simulación

In [70]:
simulator = SystemSimulator(n, s, F_dist, G_dist)
mean_crash_time = simulator.simulate(k,c,d)
print(f"Tiempo promedio hasta el colapso: {mean_crash_time[0]:.2f}")

TypeError: SystemSimulator.simulate() takes 3 positional arguments but 4 were given

## Análisis de resultados

In [52]:
crash_times = mean_crash_time[1]
plt.hist(crash_times, bins=50, density=True)
plt.xlabel('Tiempo hasta colapso')
plt.ylabel('Frecuencia')
plt.title('Distribución de tiempos de colapso')
plt.show()

IndexError: invalid index to scalar variable.

## Análisis de Sensibilidad

Investiguemos cómo varían los resultados al cambiar los parámetros del sistema.

## Modelo Matemático

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# 1. Modelo Teórico (CTMC)
def theoretical_E_T(n, s, lambda_fail, mu_repair):
    """Calcula E[T] teórico usando la fórmula de cadena de Markov"""
    E_T = 0.0
    product = 1.0
    for k in range(s + 1):
        product *= (n - k) * lambda_fail / mu_repair if k > 0 else 1
        E_T += 1 / ((n - k) * lambda_fail) * product
    return E_T



## Comparación Simulación vs Modelo

In [ ]:
# 2. Parámetros de ejemplo
n = 5
s_values = [1, 2, 3]
lambda_fail = 0.1
mu_repair = 0.5
k_simulations = 1000

# 3. Simulación y cálculos teóricos
results = {'Teórico': [], 'Simulado': [], 'Error': []}

for s in s_values:
    # Cálculo teórico
    teorico = theoretical_E_T(n, s, lambda_fail, mu_repair)
    
    # Simulación
    F = lambda size: np.random.exponential(1/lambda_fail, size)
    G = lambda: np.random.exponential(1/mu_repair)
    
    simulator = SystemSimulator(n, s, F, G)
    simulaciones = [simulator.single_run() for _ in range(k_simulations)]
    
    # Almacenar resultados
    results['Teórico'].append(teorico)
    results['Simulado'].append(np.mean(simulaciones))
    results['Error'].append(100 * abs(teorico - np.mean(simulaciones))/teorico)

# 4. Gráficas comparativas
plt.figure(figsize=(12, 6))

# Gráfico de barras comparativo
plt.subplot(1, 2, 1)
x = np.arange(len(s_values))
width = 0.35
plt.bar(x - width/2, results['Teórico'], width, label='Teórico', alpha=0.7)
plt.bar(x + width/2, results['Simulado'], width, label='Simulado', alpha=0.7)
plt.xticks(x, s_values)
plt.xlabel('Número de repuestos (s)')
plt.ylabel('E[T] (horas)')
plt.title('Comparación Teórico vs Simulado')
plt.legend()

# Gráfico de error porcentual
plt.subplot(1, 2, 2)
plt.plot(s_values, results['Error'], 'r-o')
plt.xlabel('Número de repuestos (s)')
plt.ylabel('Error Relativo (%)')
plt.title('Error entre Modelo y Simulación')
plt.grid(True)

plt.tight_layout()
plt.show()

# 5. Histograma detallado para s=2
s = 2
simulaciones = [simulator.single_run() for _ in range(k_simulations)]
teorico = theoretical_E_T(n, s, lambda_fail, mu_repair)

plt.figure(figsize=(10, 6))
plt.hist(simulaciones, bins=50, density=True, alpha=0.7, color='steelblue')
plt.axvline(teorico, color='red', linestyle='--', linewidth=2, label=f'Teórico: {teorico:.2f} h')
plt.axvline(np.mean(simulaciones), color='green', linestyle=':', linewidth=2,label=f'Simulado: {np.mean(simulaciones):.2f} h')

# Ajuste de distribución teórica
shape, loc, scale = stats.lognorm.fit(simulaciones, floc=0)
x = np.linspace(min(simulaciones), max(simulaciones), 100)
pdf = stats.lognorm.pdf(x, shape, loc, scale)
plt.plot(x, pdf, 'k-', label='Ajuste Lognormal')

plt.xlabel('Tiempo hasta colapso (horas)')
plt.ylabel('Densidad')
plt.title(f'Distribución de Tiempos de Colapso (s={s})')
plt.legend()
plt.show()

# 6. Gráfico de convergencia
convergencia = np.cumsum(simulaciones) / (np.arange(k_simulations) + 1)

plt.figure(figsize=(10, 6))
plt.plot(convergencia, label='Media Simulada')
plt.axhline(teorico, color='red', linestyle='--', label='Valor Teórico')
plt.fill_between(range(k_simulations), 
                convergencia - 1.96*np.std(simulaciones)/np.sqrt(np.arange(1, k_simulations+1)),
                convergencia + 1.96*np.std(simulaciones)/np.sqrt(np.arange(1, k_simulations+1)),
                alpha=0.2, color='blue')

plt.xlabel('Número de Simulaciones')
plt.ylabel('E[T] (horas)')
plt.title('Convergencia del Estimador')
plt.legend()
plt.grid(True)
plt.show()
